# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 1 2022 2:27PM,240979,19,ADV80003753,"AdvaGen Pharma, Ltd",Released
1,Jul 1 2022 2:23PM,240978,10,MSP211808,"Methapharm, Inc.",Released
2,Jul 1 2022 2:23PM,240978,10,MSP211809,"Methapharm, Inc.",Released
3,Jul 1 2022 2:23PM,240978,10,MSP211810,"Methapharm, Inc.",Released
4,Jul 1 2022 2:23PM,240978,10,MSP211811,"Methapharm, Inc.",Released
5,Jul 1 2022 2:23PM,240978,10,MSP211812,"Methapharm, Inc.",Released
6,Jul 1 2022 2:07PM,240977,12,HH-35602,Hush Hush,Released
7,Jul 1 2022 2:07PM,240977,12,HH-35603,Hush Hush,Released
8,Jul 1 2022 2:07PM,240977,12,HH-35605,Hush Hush,Released
9,Jul 1 2022 2:07PM,240977,12,HH-35606,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,240975,Released,17
20,240976,Released,1
21,240977,Released,5
22,240978,Released,5
23,240979,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
240975,NaN,17.0
240976,NaN,1.0
240977,NaN,5.0
240978,NaN,5.0
240979,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
240892,1.0,0.0
240918,0.0,6.0
240932,0.0,1.0
240941,0.0,1.0
240949,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
240892,1,0
240918,0,6
240932,0,1
240941,0,1
240949,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,240892,1,0
1,240918,0,6
2,240932,0,1
3,240941,0,1
4,240949,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,240892,1,
1,240918,,6
2,240932,,1
3,240941,,1
4,240949,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 1 2022 2:27PM,240979,19,"AdvaGen Pharma, Ltd"
1,Jul 1 2022 2:23PM,240978,10,"Methapharm, Inc."
6,Jul 1 2022 2:07PM,240977,12,Hush Hush
11,Jul 1 2022 1:58PM,240976,10,Emerginnova
12,Jul 1 2022 1:46PM,240975,10,ISDIN Corporation
29,Jul 1 2022 1:44PM,240971,10,ISDIN Corporation
66,Jul 1 2022 12:59PM,240967,10,Eywa Pharma Inc.
67,Jul 1 2022 12:34PM,240960,10,Bio-PRF
83,Jul 1 2022 12:31PM,240962,20,"Exact-Rx, Inc."
102,Jul 1 2022 12:23PM,240959,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 1 2022 2:27PM,240979,19,"AdvaGen Pharma, Ltd",,1
1,Jul 1 2022 2:23PM,240978,10,"Methapharm, Inc.",,5
2,Jul 1 2022 2:07PM,240977,12,Hush Hush,,5
3,Jul 1 2022 1:58PM,240976,10,Emerginnova,,1
4,Jul 1 2022 1:46PM,240975,10,ISDIN Corporation,,17
5,Jul 1 2022 1:44PM,240971,10,ISDIN Corporation,,37
6,Jul 1 2022 12:59PM,240967,10,Eywa Pharma Inc.,,1
7,Jul 1 2022 12:34PM,240960,10,Bio-PRF,5,11
8,Jul 1 2022 12:31PM,240962,20,"Exact-Rx, Inc.",9,10
9,Jul 1 2022 12:23PM,240959,20,"ACI Healthcare USA, Inc.",4,7


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 1 2022 2:27PM,240979,19,"AdvaGen Pharma, Ltd",1,
1,Jul 1 2022 2:23PM,240978,10,"Methapharm, Inc.",5,
2,Jul 1 2022 2:07PM,240977,12,Hush Hush,5,
3,Jul 1 2022 1:58PM,240976,10,Emerginnova,1,
4,Jul 1 2022 1:46PM,240975,10,ISDIN Corporation,17,
5,Jul 1 2022 1:44PM,240971,10,ISDIN Corporation,37,
6,Jul 1 2022 12:59PM,240967,10,Eywa Pharma Inc.,1,
7,Jul 1 2022 12:34PM,240960,10,Bio-PRF,11,5
8,Jul 1 2022 12:31PM,240962,20,"Exact-Rx, Inc.",10,9
9,Jul 1 2022 12:23PM,240959,20,"ACI Healthcare USA, Inc.",7,4


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 1 2022 2:27PM,240979,19,"AdvaGen Pharma, Ltd",1,
1,Jul 1 2022 2:23PM,240978,10,"Methapharm, Inc.",5,
2,Jul 1 2022 2:07PM,240977,12,Hush Hush,5,
3,Jul 1 2022 1:58PM,240976,10,Emerginnova,1,
4,Jul 1 2022 1:46PM,240975,10,ISDIN Corporation,17,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 1 2022 2:27PM,240979,19,"AdvaGen Pharma, Ltd",1.0,NaN
1,Jul 1 2022 2:23PM,240978,10,"Methapharm, Inc.",5.0,NaN
2,Jul 1 2022 2:07PM,240977,12,Hush Hush,5.0,NaN
3,Jul 1 2022 1:58PM,240976,10,Emerginnova,1.0,NaN
4,Jul 1 2022 1:46PM,240975,10,ISDIN Corporation,17.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 1 2022 2:27PM,240979,19,"AdvaGen Pharma, Ltd",1.0,0.0
1,Jul 1 2022 2:23PM,240978,10,"Methapharm, Inc.",5.0,0.0
2,Jul 1 2022 2:07PM,240977,12,Hush Hush,5.0,0.0
3,Jul 1 2022 1:58PM,240976,10,Emerginnova,1.0,0.0
4,Jul 1 2022 1:46PM,240975,10,ISDIN Corporation,17.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2650565,132.0,29.0
12,240977,5.0,0.0
16,722728,12.0,1.0
19,240979,1.0,0.0
20,722877,37.0,18.0
21,240949,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2650565,132.0,29.0
1,12,240977,5.0,0.0
2,16,722728,12.0,1.0
3,19,240979,1.0,0.0
4,20,722877,37.0,18.0
5,21,240949,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,132.0,29.0
1,12,5.0,0.0
2,16,12.0,1.0
3,19,1.0,0.0
4,20,37.0,18.0
5,21,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,132.0
1,12,Released,5.0
2,16,Released,12.0
3,19,Released,1.0
4,20,Released,37.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Executing,29.0,0.0,1.0,0.0,18.0,0.0
Released,132.0,5.0,12.0,1.0,37.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Executing,29.0,0.0,1.0,0.0,18.0,0.0
1,Released,132.0,5.0,12.0,1.0,37.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Executing,29.0,0.0,1.0,0.0,18.0,0.0
1,Released,132.0,5.0,12.0,1.0,37.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()